In [1]:
# Import Libaries and Tools
import os
import sys
from astropy.io import fits
import sunpy.map
from aiapy.calibrate import register, update_pointing
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import imageio
import glob

# ACWE utilities
# Root directory of the project
ROOT_DIR = os.path.abspath("../../../../")

# Import ACWE Tools
sys.path.append(ROOT_DIR)
from ACWE_python_fall_2023 import acweSaveSeg_v5, acweRestoreScale
from DatasetTools import DataManagmentTools as dmt

plt.rcParams.update({'image.interpolation': 'None'})

In [ ]:
# Dataset

# Carrington Rotation
CR = 'CR2101'

# ACWE Parameters 
acweChoice = '193'

# Dataset
dataset = '/home/jgra/Coronal Holes/newDataset/'

# ACWE Folders
folderBase = '/mnt/coronal_holes/Paper 2/Code 02 Observations/HMI_Experiments/'
newACWE = folderBase + '/TestEvolutionMethods/Scaled/Uni/'
oldACWE = folderBase + '/TestEvolutionMethods/SeedTransfer/Uni/'

# Stats file
datasetfile = 'Results/'+CR+'.csv.Uni.SeedTransfer.npz'

In [ ]:
# Open File and Prepare for ACWE
def openAIA(filename):
    
    # Extract Image and Header Data
    hdulist = fits.open(filename)
    hdulist.verify('silentfix') # no clue why this is needed for successful data read
    h = hdulist[1].header
    J = hdulist[1].data
    hdulist.close()
    
    # Update to Level 1.5 Data Product
    if h['LVL_NUM'] < 1.5:
        m = sunpy.map.Map((J,h))    # Create Sunpy Map
        m = update_pointing(m)      # Update Header based on Latest Information
        m_registrered = register(m) # Recenter and rotate to Solar North
        I = m_registrered.data
        # Undo Keword Renaming
        H = dict()
        for k in m_registrered.meta.keys(): 
            H[k.upper()] = m_registrered.meta[k] 
    # Skip if already Level 1.5
    else:
        I = J*1
        H = dict(h)
        
        # Prepare Display Version
    Idsp = np.clip(I,20,2500)
    Idsp = np.log10(Idsp)
    Idsp = Idsp - np.min(Idsp)
    Idsp = Idsp/np.max(Idsp)
    
    return I,Idsp,H

In [ ]:
# Get Stats

# open and retreave data
data = np.load(datasetfile, allow_pickle=True)
lst = data.files

# Get inital list of stats
alphas = data[lst[0]]
IOU    = data[lst[1]]
SSIM   = data[lst[2]]
GCE    = data[lst[3]]
LCE    = data[lst[4]]

In [ ]:
# Get Images and Segmentations

# Get List of Images
euvImages = sorted(glob.glob(dataset + CR + '/*/*.' + acweChoice + '.*'))

# Get List of Segmentaitons
newACWEsegs = sorted(glob.glob(newACWE + CR + '/*/*.' + acweChoice + '.*'))
oldACWEsegs = sorted(glob.glob(oldACWE + CR + '/*/*.' + acweChoice + '.*'))

In [ ]:
# Cycle Through Dataset

for i in range(len(IOU)):
    
    # Check IOU
    if IOU[i,0,1] <= 0.5:
        
        # Get corrispoinding data
        thisIOU    = IOU[i,0,1]
        euvImage   = euvImages[i]
        newACWEseg = newACWEsegs[i]
        oldACWEseg = oldACWEsegs[i]
        
        # Open Image
        _,Idsp,_ = openAIA(euvImage)
        
        # Open Segmentations
        _,newAH,newSeg = acweSaveSeg_v5.openSeg(newACWEseg)
        _,oldAH,oldSeg = acweSaveSeg_v5.openSeg(oldACWEseg)
        oldSeg = acweRestoreScale.upscale(oldSeg,oldAH)
        newSeed   = newAH['INIT_MASK']
        oldSeed   = oldAH['INIT_MASK']
        oldSeed   = acweRestoreScale.upscale(oldSeed,oldAH)
        
        
        # Display Results
        plt.figure(figsize=[15,17],dpi=300)
        plt.rcParams.update({'font.size': 22})
        title = os.path.basename(euvImage) + '\nFinal Seg IOU:' + str(thisIOU)
        plt.suptitle(title)
        plt.subplot(3,3,2)
        plt.imshow(np.flip(Idsp,axis=0),cmap='gray')
        plt.title('EUV Observation')
        plt.axis(False)
        plt.subplot(3,3,4)
        plt.imshow(np.flip(oldSeed,axis=0))
        plt.title('Seed: 1/8 Scale')
        plt.axis(False)
        plt.subplot(3,3,5)
        plt.imshow(np.flip(newSeed,axis=0))
        plt.title('Seed: Full Scale')
        plt.axis(False)
        plt.subplot(3,3,6)
        plt.imshow(np.flip(oldSeed.astype(int)-newSeed.astype(int),axis=0),cmap='gray',vmin=-1,vmax=1)
        plt.title('Seed: 1/8 - Full')
        plt.axis(False)
        plt.subplot(3,3,7)
        plt.imshow(np.flip(oldSeg,axis=0))
        plt.title('Seg: 1/8 Scale')
        plt.axis(False)
        plt.subplot(3,3,8)
        plt.imshow(np.flip(newSeg,axis=0))
        plt.title('Seg: Full Scale')
        plt.axis(False)
        plt.subplot(3,3,9)
        plt.imshow(np.flip(oldSeg.astype(int)-newSeg.astype(int),axis=0),cmap='gray',vmin=-1,vmax=1)
        plt.title('Seg: 1/8 - Full')
        plt.axis(False)
        plt.show()